# Caribbean Network Complexity

This is an indicator for the 2023 Virgin Islands/Puerto Rico blueprint.

Created by Amy Keister, last run by Amy Keister 15 February 2023. It took 5 minutes to run.

In [90]:
import os
import arcpy

In [91]:
import time
start = time.time()

In [92]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace = r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanNetworkComplexity\CarribeanNet.gdb"

In [93]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanNetworkComplexity\CaribbeanNetworkComplexity.tif"

In [94]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

In [95]:
# define inputs
# FEMA floodplain
FP = r"F:\GIS_DATA\WaterResources\VIPR_FEMA_FLOODPLAIN\PRVI_Floodplain_5070.shp"
# SARP funtional network lines
FNL= r"F:\GIS_DATA\WaterResources\NHD\VIPR_FromBrendan\region21_dams_networks_1_24_2023\region21_dams_networks.gdb\region21_dams_networks"
# SARP functional network line segments
FNLS= r"F:\GIS_DATA\WaterResources\NHD\VIPR_FromBrendan\region21_dams_networks_1_24_2023\region21_dams_segments.gdb\region21_dams_segments"
# NHDPLusHR catchments from national download
catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NHDPlusCatchment"
# NHDPlusHR flowlines from national download
flow= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NetworkNHDFlowline"
# there is a large missing catchment on the west coast of PR. I made this to fill it in
missing= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\MissingCatchPR.shp"

### Start Analysis

In [96]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [97]:
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\4_Indicators\CaribbeanNetworkComplexity\CarribeanNet.gdb


### Bring in SARP high res data

In [9]:
# make a copy of Brendan's data for editing
arcpy.management.CopyFeatures(FNL, "FNL", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNetworkComplexity\\CarribeanNet.gdb\\FNL'>

In [10]:
# reproject Brendan's data

# double check with Brendan that this is correct

arcpy.management.Project("FNL", "FNLrepro", 'PROJCS["NAD_1983_Contiguous_USA_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-96.0],PARAMETER["Standard_Parallel_1",29.5],PARAMETER["Standard_Parallel_2",45.5],PARAMETER["Latitude_Of_Origin",23.0],UNIT["Meter",1.0]]', None, sr)

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNetworkComplexity\\CarribeanNet.gdb\\FNLrepro'>

In [11]:
# convert Brendan's high res linework to raster, using the "sizes" field, which represents network complexity
# before I calaculated this at the original NHDPlus raster cell size, but I don't think that matters too much
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent):
    arcpy.conversion.FeatureToRaster(in_features="FNLrepro", field="sizeclasses", out_raster="FNLsizes", cell_size=Rextent)

In [12]:
# add in missing catchment on west side of Puerto Rico
# I had created this catchment to fill in the gap. It is a weird error with sevearl narrow "catchments" 
# circling a donut hole

with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.Union([catch, missing], "catch", "ALL", None, "NO_GAPS")

In [13]:
# use zonal statistics to assign "sizes" value from Brendan's linework to the national NHDPlus HR catchments
# we need to do this to turn the linework into an area. The network complexity is calculated by Brendan
# using line network analysis, for the blueprint, we need to assign those lines back to an area, we start by
# assigning them back to the catchment that intersects with the lines.
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ZonalStatistics("catch", "OBJECTID", "FNLsizes", "MAJORITY", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZBcatSizesMaj")

### Make FEMA floodplain layer

In [14]:
# make a copy of the flooplain vector
arcpy.management.CopyFeatures(FP, "FP", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNetworkComplexity\\CarribeanNet.gdb\\FP'>

In [15]:
# add field to floodplain to prepare to convert to raster
arcpy.management.CalculateField("FP", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNetworkComplexity\\CarribeanNet.gdb\\FP'>

In [16]:
# convert floodplain to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.PolygonToRaster("FP", "raster", "FPr", "CELL_CENTER", "NONE", Rextent, "BUILD")

### Add in buffered flow lines to improve the floodplain layer

In [17]:
# make a copy of the flowlines in the study area
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures(flow, "flow", '', None, None, None)

In [18]:
# extract the stream/river lines
# need to check with Kat to see which lines she uses
arcpy.analysis.Select("flow", "RS", "ftype IN (460, 558)")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNetworkComplexity\\CarribeanNet.gdb\\RS'>

In [19]:
# add field to prepare to convert to raster
arcpy.management.CalculateField("RS", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNetworkComplexity\\CarribeanNet.gdb\\RS'>

In [20]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("RS", "raster", "RSr", 30)

In [21]:
# use and expand to "buffer" the streams by 1 cell, this should create total stream width of ~90 meters
out_raster = arcpy.sa.Expand("RSr", 1, [1], ""); out_raster.save("RSrExpand1")

### Combine FEMA floodplains and buffered flow lines to make floodplain layer

In [22]:
# mosaic the floodplain and buffered rasters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.MosaicToNewRaster("FPr;RSrExpand1", OutWorkspace, "FPflowa", None, "8_BIT_UNSIGNED", None, 1, "MAXIMUM", "FIRST")

In [23]:
# Take times 1 and re calculate extent because the output is wonky
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster= arcpy.sa.Times("FPflowa", 1); out_raster.save("FPflow")

### Prep data to make a zero class
zeros will be areas where there are NHD data, and NoData will be areas where there is not NHD data

In [24]:
# add field to the catchment layer to prepare to convert to raster
arcpy.management.CalculateField("catch", "raster", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNetworkComplexity\\CarribeanNet.gdb\\catch'>

In [25]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("catch", "raster", "catch1", 30)

### Clip the zonal statistics layer to floodplain and buffered flow lines to make a preliminary network complexity layer

In [26]:
# use conditional statement to limit results to the floodplain layer
# the result is the prelimiary network complexity layer
# this layer has "holes" in it where Brendan has had to remove the "loops" of braided streams
# so he can do the network analysis
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Con("FPflow", "ZBcatSizesMaj", "", where_clause="Value = 1"); out_raster.save("prelimNC")

### Deal with missing catchments inside floodplain

because Brendan has to remove loops in the network some catchments inside the floodplain do not intersect with the linework that Brendan has calculated network complexity on. These should be at least a value of 1, because they are in the floodplain. Some should have values higher than that. Normally these are small catchments in braided stream networks in large floodplains. They may be separated from the floodplain by a natural levee, but during flood stage, they should be connected to the flood plain around them.

In [27]:
# give all areas that didn't intersect with Brendan's data a crazy value we can keep track of
out_raster = arcpy.sa.Reclassify("prelimNC", "Value", "NODATA 20", "DATA"); out_raster.save("prelimNC20")

In [28]:
# take catchment layer times floodplain layer to help find missing catchments that don't intersect 
# Brendan's linework. We don't want to map a floodplain anywhere there 
# isn't also NHD catchments
out_raster= arcpy.sa.Times("catch1", "FPflow"); out_raster.save("FPflowcatch")

In [29]:
# clip back down to floodplain/NHD catchment area
# areas with the crazy value in this raster are catchments that should be at least a value of 1
out_raster= arcpy.sa.Times("prelimNC20", "FPflowcatch"); out_raster.save("prelimNC20clip")

In [30]:
# pull those areas that are in the floodplain, that have NHD catchment data, and that do not intersect 
# brendan linework (crazy value) as a unique raster
out_raster = arcpy.sa.Con("prelimNC20clip", "1", "", where_clause="Value = 20"); out_raster.save("missing")

In [31]:
# expand by one pixel
out_raster = arcpy.sa.Expand("missing", 1, [1], ""); out_raster.save("missingExpnd1")

In [32]:
# use a region group to identify individual missing areas
out_raster = arcpy.sa.RegionGroup("missingExpnd1", "FOUR", "WITHIN", "ADD_LINK", None); out_raster.save("missingExpnd1rg")

In [33]:
# calculate maximum value in the preliminary network complexity that intsersect expanded missing areas
# useing zonal statistics
out_raster = arcpy.sa.ZonalStatistics("missingExpnd1rg", "Value", "prelimNC", "MAXIMUM", "DATA", "CURRENT_SLICE", 90); out_raster.save("MissingZstatPncMax")


In [34]:
# use a conditional statement to assign maximum nearby values to the missing areas in the preliminary network complexity
out_raster = arcpy.sa.Con("prelimNC20clip", "MissingZstatPncMax", "prelimNC20clip", where_clause="Value = 20"); out_raster.save("prelimNCfill")

In [35]:
# reclassify to make NoData zero
out_raster = arcpy.sa.Reclassify("prelimNCfill", "Value", "NODATA 0;1 1;2 2;3 3;4 4", "DATA"); out_raster.save("prelimNCfill0")

In [36]:
# clip back down to NHD catchment area
out_raster= arcpy.sa.Times("prelimNCfill0", "catch1"); out_raster.save("indicator")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to SE extent, clip and export to SA extent

In [37]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 7:
        return '7 = 7 connected stream classes'
    elif value == 6:
        return '6 = 6 connected stream classes'
    elif value == 5:
        return '5 = 5 connected stream classes'
    elif value == 4:
        return '4 = 4 connected stream classes'
    elif value == 3:
        return '3 = 3 connected stream classes'
    elif value == 2:
        return '2 = 2 connected stream classes'
    elif value == 1:
        return '1 = 1 connected stream class'
    elif value == 0:
        return '0 = Not identified as a floodplain'
"""
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanNetworkComplexity\\CarribeanNet.gdb\\indicator'>

In [38]:
# add and calculate description field to hold indicator values
#arcpy.management.CalculateField("indicator", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

In [98]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 60
	if Value == 3:
		return 114
	if Value == 2:
		return 175
	if Value == 1:
		return 215
	else:
		return 255
		
def Reclass2(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 147
	if Value == 3:
		return 172
	if Value == 2:
		return 193
	if Value == 1:
		return 213
	else:
		return 255
		
def Reclass3(Value):
	if Value == 5:
		return 0
	if Value == 4:
		return 194
	if Value == 3:
		return 209
	if Value == 2:
		return 221
	if Value == 1:
		return 233
	else:
		return 255
"""
# calculate Red field
arcpy.management.CalculateField("indicator", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'indicator'>

In [99]:
# clip to extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("indicator1")

In [100]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("indicator1", Out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [42]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

246.2326853275299
